In [16]:
import tensorflow as tf
import os
from matplotlib import pyplot as plt
from PIL import Image
import numpy as np

%matplotlib inline

In [24]:
!cd ../models/research/object_detection

data			   models			 Test the result.ipynb
export_inference_graph.py  ssd_mobilenet_v1_coco.config  train.py
fine_tuned_model	   test_images


In [22]:
!export PYTHONPATH=$PYTHONPATH:`pwd`:`pwd`/slim

In [23]:
from utils import label_map_util

from utils import visualization_utils as vis_util

ImportError: No module named utils

In [4]:
class TrafficLightClassifier(object):
    def __init__(self):
        PATH_TO_MODEL = 'fine_tuned_model/frozen_inference_graph.pb'
        self.detection_graph = tf.Graph()
        with self.detection_graph.as_default():
            od_graph_def = tf.GraphDef()
            # Works up to here.
            with tf.gfile.GFile(PATH_TO_MODEL, 'rb') as fid:
                serialized_graph = fid.read()
                od_graph_def.ParseFromString(serialized_graph)
                tf.import_graph_def(od_graph_def, name='')
            self.image_tensor = self.detection_graph.get_tensor_by_name('image_tensor:0')
            self.d_boxes = self.detection_graph.get_tensor_by_name('detection_boxes:0')
            self.d_scores = self.detection_graph.get_tensor_by_name('detection_scores:0')
            self.d_classes = self.detection_graph.get_tensor_by_name('detection_classes:0')
            self.num_d = self.detection_graph.get_tensor_by_name('num_detections:0')
        self.sess = tf.Session(graph=self.detection_graph)
        
        
    def get_classification(self, img):
        # Bounding Box Detection.
        with self.detection_graph.as_default():
            # Expand dimension since the model expects image to have shape [1, None, None, 3].
            img_expanded = np.expand_dims(img, axis=0)  
            (boxes, scores, classes, num) = self.sess.run(
                [self.d_boxes, self.d_scores, self.d_classes, self.num_d],
                feed_dict={self.image_tensor: img_expanded})
        return boxes, scores, classes, num

In [7]:
PATH_TO_CKPT = 'fine_tuned_model/frozen_inference_graph.pb'
PATH_TO_LABELS = os.path.join('data', 'sim_label_map.pbtxt')
NUM_CLASSES = 3

In [8]:
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

In [ ]:
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

In [9]:
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

In [17]:
test_img_path = 'test_images/out00002.png'
test_img = Image.open(test_img_path)
img = load_image_into_numpy_array(test_img)

In [18]:
tlc  =TrafficLightClassifier()

In [19]:
tlc.get_classification(img)

(array([[[0.        , 0.8369585 , 0.33774036, 1.        ],
         [0.        , 0.88024783, 0.31456298, 1.        ],
         [0.        , 0.4683299 , 0.33946016, 0.70994174],
         [0.        , 0.5731171 , 0.33047912, 0.7995612 ],
         [0.        , 0.8132633 , 0.29853594, 0.9979738 ],
         [0.        , 0.4683299 , 0.33946016, 0.70994174],
         [0.        , 0.6773695 , 0.3055301 , 0.8953929 ],
         [0.        , 0.5731171 , 0.33047912, 0.7995612 ],
         [0.        , 0.38863432, 0.32779253, 0.60894424],
         [0.76132274, 0.69450164, 0.9463724 , 1.        ],
         [0.        , 0.20114483, 0.27187282, 0.3886726 ],
         [0.68044674, 0.03782097, 1.        , 0.24524337],
         [0.        , 0.2412239 , 0.87280226, 0.6719731 ],
         [0.        , 0.6253805 , 0.8788773 , 1.        ],
         [0.        , 0.3013991 , 0.2933914 , 0.5060641 ],
         [0.56236076, 0.64166355, 1.        , 1.        ],
         [0.        , 0.38863432, 0.32779253, 0.60894424